In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

25/02/26 02:14:43 WARN Utils: Your hostname, spark2 resolves to a loopback address: 127.0.1.1; using 192.168.69.5 instead (on interface enp0s1)
25/02/26 02:14:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/02/26 02:14:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.version

'3.3.2'

In [3]:
df = spark.read.parquet('data/yellow_tripdata_2024-10.parquet')

In [8]:
df.repartition(4).write.parquet('data/partitions')

In [4]:
from pyspark.sql import functions as f

df = df \
    .withColumn('pickup_date', f.to_date(df.tpep_pickup_datetime))

In [5]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|pickup_date|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1

In [6]:
df \
    .where(df.pickup_date == '2024-10-15') \
    .count()

128893

In [7]:
def trip_time_mins(pickup_datetime, dropoff_datetime):
    return (dropoff_datetime - pickup_datetime).total_seconds() / 60

from pyspark.sql import types as t
trip_time_udf = f.udf(trip_time_mins, returnType=t.FloatType())

In [9]:
df = df \
    .withColumn('trip_duration_mins', trip_time_udf(df.tpep_pickup_datetime, df.tpep_dropoff_datetime))
df \
    .agg(f.max(df.trip_duration_mins)) \
    .show()

+-----------------------+
|max(trip_duration_mins)|
+-----------------------+
|               9757.066|
+-----------------------+



In [15]:
df_zones = spark.read \
    .option("header", "true") \
    .csv('data/taxi_zone_lookup.csv')

df_zones.createOrReplaceTempView('zones')
df.createOrReplaceTempView("trip_data")

In [30]:
spark.sql("""
SELECT
    td.PUlocationID,
    z.Zone,
    COUNT(*) as trip_count
FROM
    trip_data AS td
INNER JOIN
    zones AS z
ON
    td.PULocationID = z.LocationID
GROUP BY td.PUlocationID, z.Zone
ORDER BY trip_count ASC
LIMIT 1
""").head(1)

[Row(PUlocationID=105, Zone="Governor's Island/Ellis Island/Liberty Island", trip_count=1)]